# The ECPV data model
+ ECPV stands for Entity-Context-Property-Value
+ ECPV is a data model for metadata concerning a set of entities. It is a subset of EAV (Entity-Attribute-Value).
+ The ECPV model has similar goals to [RDF] [rdf], but different design and implementation. It is fact based and not assertion based.  

## Basic Data Pattern
1. A list (or Array) of entities. This list of entities can be comprised of any column-modeled facts, as long as:
    a. Each row is distinct. An entity must be unique in the list. 
    b. There is a way to designate the ECPV file or object itself, i.e there is a place holder or way to describe the file or object containing the descriptions. (self)
2. A list or set of contexts such that:
    a. Each context has a property ('obligate contexts') that communicates any extra data tables required to describe the context itself. These will be other contexts. 
    b. The understood or implicit context is the EPCV file or object (self)
3. A list of property or attribute names that is:
    a. comprised of a list of strings (naming restrictions can be context dependent). Each property name is a single string.  
    b. When taken with a context and entity form a unique identifier (independent variable)
4. A list of property or attribute values 
    a. That is associated with one and only one ECP triple (single valued). That is ECP can be thought of a hash table (name) for v.

## Closed, Open, Abstract
An ECPV model is called __closed__ if all the entities contained in the Entitiy list are availible to the interptreting agent (program). For example, if all the entitites are files located on the computer reading the ECPV and not blocked by permissions.
An ECPV model is called __open__ if all the entities are electronic resources, but may or may not be resolvable. Websites or files on other computers
An ECPV model is called __abstract__ if the entities do not represent electronic resources  

### The E in ECPV
1. E is for entity, and it can refer to a table of records or the file that the description is in. 
2. If E is a table of records then it is assumed to be in the relational database 1st normal form, or simply that each row is composed of a fixed number of columns (no ragged lists) and each row is uniquely identifiable. Different Entity lists or tables can use a context to provide a description.
3. If the entity is not implicit ( a list or outside of the file itself) there must be a reserved place for describing the entity that contains the description. (reserved idea of self description)

### The C in ECPV
1. C is for context. A context is the knowledge system used to describe the entity, it consists of all of the properties used in that specific context and a set of facts about those properties.
2. The facts describing a property are stored in obligate contexts. A set of obligate contexts can be thought of as a free form schema for a particular context, effectively defining all facts about the ECP combination. Obligate is a reserved idea. In addition, default is a reserved idea, that is for obligate-contexts that do not have properties defined they are assumed to be default is specified. 
3. If a context is not specified it is ambigous, but assumed to be first the file that the description occurs in, and second (if needed) the program that uses the file the description is in and third the program that authored the file the description is in.

### The P in ECPV
1. P is for property. It is a character encoded string representing some aspect to be described. It can be thought of as an attribute name, but can represent relationships. 
 

### The V in ECPV
1. V is for value. It can be anything that can be represented electronically and should follow the descriptions found in the obligate contexts. 
2. V is single valued for a particular ECP combination. That is ECP can only return one thing at any given instant in time. 




# Implementation Strategies

## RDBMS strategy
1. Define an entity table using a number of columns needed to uniquely identify an object. Create an artifical key that will be used internally.
2. Create a context (named for the knowledge system or program that uses it) table. This table should have a list of properties (propert name) and any needed fields such as (type,etc),
3. Create a description table or tables with a Foriegn Key from both the entity table and context table. Use the context table to type the property=value pairs.

## XML Strategy I
1. Create an XML file with a root element such as `<Entity_Registry>` and sub elements for each Object or Entity with attributes names 
```xml
<Entity url="http://localhost:8888/notebooks/PyCharmProjects/Jupyter-Notebooks/ECPV_Example_20160518_001.ipynb#" date_added='2016-05-19'/>``` 
.
2. Create a second file that has identical elements as 1. with the Entity elements as complex elements.
```xml
<Metadata_Registry><Entity url="http://localhost:8888/notebooks/PyCharmProjects/Jupyter-Notebooks/ECPV_Example_20160518_001.ipynb#" date_added='2016-05-19'> <Property_1>Value_1</Proerty_1></Entity></Metadata>```
3. Create a XSD or DTD that satisfies the needs of the obligate-contexts

## Python

In [8]:
import re

In [5]:
class ContextModel():
    """Context model for ECPV"""
    def __init__(file_path=None,**options):
        """Intialize the ContextModel Class, open it if it is a file or create a new one"""
        defaults={"properties":[],'obligates':None,'name':None}
        self.options={}
        for key,value in defaults.iteritems():
            self.options[key]=value
        for key,value in options.iteritems():
            self.options[key]=value
        attributes=['properties','obligates','name']
        for attribute in attributes:
            self.__dict__[attribute]=self.options[attribute]
            
    def add_obligate(self,obligate_context_name, obligate_context_default=None,obligate_fact_dictionary=None):
        """Adds an obligate context (Metadata table about the properties)"""
        self.obligates.append(obligate_context_name)
        self.obligate_defaults.append(obligate_context_default)
        if obligate_fact_dictionary is not None:
            self.__dict__[obligate_context_name]=obligate_fact_dictionary
                
    def add_property(property_name,obligate_values=None):
        """Adds a property to the context, if obligate values is a list of length equal to self.obligates 
        it assumes them to be in order. If it is a dictionary, the key should be the obligate context name and 
        the value should be the value. The new obligate dictionary will be self.obligate_context_name[property_name]
        = value. Only adds values where they are defined.  """
        try:
            self.properties.append(property_name)
            if obligate_values:
                if type(obligate_values) is ListType and len(obligate_values)==len(self.obligates): 
                    for index,obligate in enumerate(self.obligates[:]):
                        self.__dict__[obligate][property_name]=obligate_values[index]
                elif type(obligate_values) is DictType:
                    for obligate,value in obligate_values.iteritems():
                        self.__dict__[obligate][property_name]=value

        except:
            print("Could not add property {0}".format(property_value))

            
            
            # a python class implementation
# simplest form is a list-list-dictionary where for each index in the top most list there is a context that is then
# a list of property:value pairs or actually just a dictionary with a string for a key and self/self is understood
class ECPV():
    """Test class for ECPV data model"""
    def __init__(self):
        """Intializes the ECPV data model"""
        self.entities=['self']
        self.contexts=['self']
        self.property_names=['color']
        key=self.entities[0]+"/"+self.contexts[0]+"/"+self.property_names[0]
        self.descriptions={key:"Red"}

new=ECPV()
new.descriptions
        

{'self/self/color': 'Red'}

In [ ]:
xml_string="<Entity name='self'><Context name='self'><Color>Red</Color></Context><Entity>"

In [6]:
header="""Date of test: 2015 March 11 8.0 - 12.0 GHz
Directory:  N:\artifact\X1\x12zhz.137\Calibrations\2015 March 11 8.0 - 12.0 GHz
File name:  Raw_15Mar11_14.25-3.txt

Frequency list (GHz):  8.000	9.000	10.000	11.000	12.000

Customer: Northrup Grumman Space Tech
Address:  2477 Manhattan Beach Blvd
Redondo Beach, CA 90278

DUT:  x12zhz.137
Operator: DG
Remarks: DUT 811454 Cal #23

Amb Std: THTZ.010 on port 1
Cryo Std: PS2ZCZ.200 on port 2
Chk Std: x12zhz.137 on port 4
DUT 2: 811454 on port 6



++++++++++++++++++++++++++++++++++++
DUT 1: x12zhz.137 on port 4
++++++++++++++++++++++++++++++++++++"""


In [13]:
header_line_list=header.splitlines()
header_pattern=re.compile('(?P<header_name>^.*?):(?P<header_value>.*)')
header_dictionary={}
for i,line in enumerate(header_line_list):
    try:
        match_list=re.search(header_pattern,line)
        if match_list:
            key=match_list.group('header_name').replace(' ','_')
            key=key.translate(None,'()~!#$%^&*{}[]<>?;')
            key=key.strip()
            value=match_list.group('header_value')
            if re.match('Address',key):
                value=value+', '+header_line_list[i+1]
            header_dictionary[key]=value
    except:
        pass

In [17]:
header_dictionary["Address"]

'  2477 Manhattan Beach Blvd, Redondo Beach, CA 90278'